## Adam Packer and Aaron Newman (08-apacker-newmanar)
## SIADS 593 Final Report
## Winter 2023
## Setup Notebook

# 0.1 - Import necessary modules

In [113]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')

# 1.0 - Travel Data

### 1.1 - Obtaining and reading in the raw data

We obtained our air travel data from the Bureau of Transportation Statistics at the following link:<br>
https://transtats.bts.gov/Fields.asp?gnoyr_VQ=FIL

The webpage there gives an explanation of the fields and links to download the data in .csv format.

In [114]:
# read in all the travel data
# assumes that user has extracted the relevant files from downloaded zips)
df_bts_2020 = pd.read_csv('2020_T_T100D_MARKET_US_CARRIER_ONLY.csv')
df_bts_2021 = pd.read_csv('2021_T_T100D_MARKET_US_CARRIER_ONLY.csv')
df_bts_2022 = pd.read_csv('2022_T_T100D_MARKET_US_CARRIER_ONLY.csv')

# NOTE: 2022 data is only through November.  December 2022 data is not available at the time of this writing.

In [115]:
# collect into a single dataframe
df_bts_region = pd.concat([df_bts_2022, df_bts_2021, df_bts_2020])

In [116]:
df_bts_region.head()

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,UNIQUE_CARRIER_NAME,UNIQUE_CARRIER_ENTITY,REGION,CARRIER,...,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS
0,0.0,0.0,0.0,0.0,09Q,21161,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",01130,D,09Q,...,"Alexandria, LA",LA,22,Louisiana,72,2022,1,3,1,L
1,0.0,0.0,0.0,0.0,09Q,21161,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",01130,D,09Q,...,"Phoenix, AZ",AZ,4,Arizona,81,2022,1,1,1,L
2,0.0,0.0,0.0,0.0,09Q,21161,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",01130,D,09Q,...,"Bullhead City, AZ",AZ,4,Arizona,81,2022,1,1,1,L
3,0.0,0.0,0.0,0.0,09Q,21161,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",01130,D,09Q,...,"Bullhead City, AZ",AZ,4,Arizona,81,2022,1,2,1,L
4,0.0,0.0,0.0,0.0,09Q,21161,"Swift Air, LLC d/b/a Eastern Air Lines d/b/a E...",01130,D,09Q,...,"Baton Rouge, LA",LA,22,Louisiana,72,2022,1,3,1,L


In [117]:
df_bts_region.shape

(664691, 36)

### 1.2 - Extract and organize the travel data we are interested in

In [118]:
# keep only the columns we are interested in, where there are passengers to count
df_bts_region = df_bts_region[['YEAR', 'MONTH', 'ORIGIN_CITY_MARKET_ID', 'DEST_CITY_MARKET_ID',
                               'UNIQUE_CARRIER', 'PASSENGERS']][df_bts_region.PASSENGERS>0]
# rename the columns
df_bts_region.columns = ['year', 'month', 'origin', 'dest', 'carrier', 'passengers']

# organize the data
df_bts_region = df_bts_region.groupby(['year', 'month', 'origin', 'dest', 'carrier']).sum().reset_index()

# reformat the date data
df_bts_region['date'] = pd.to_datetime(dict(year=df_bts_region.year, month=df_bts_region.month, day=1))
df_bts_region.drop(['year', 'month'], axis=1, inplace=True)

In [119]:
df_bts_region.head()

,origin,dest,carrier,passengers,date
0,30006,30056,2O,4.0,2020-01-01
1,30007,31401,I4,631.0,2020-01-01
2,30007,32523,J5,1.0,2020-01-01
3,30007,34828,J5,55.0,2020-01-01
4,30009,30009,2E,11.0,2020-01-01


In [120]:
df_bts_region.shape

(473065, 5)

### 1.3 - Aggregate passenger counts by location

Get number of passengers departing on [date], via [carrier], from [origin].<br>
Get number of passengers arriving on [date], via [carrier], at [destination].

In [121]:
# aggregate passenger counts by date, carrier, origin
df_pass_from = df_bts_region.reset_index().groupby(['date', 'carrier', 'origin']).sum()['passengers'].to_frame()
df_pass_from = df_pass_from.reset_index().rename(columns={'origin': 'bts_loc_code', 'passengers': 'p_origin'})
df_pass_from = df_pass_from.set_index(['date', 'carrier', 'bts_loc_code'])

# aggregate passenger counts by date, carrier, dest
df_pass_to = df_bts_region.reset_index().groupby(['date', 'carrier', 'dest']).sum()['passengers'].to_frame()
df_pass_to = df_pass_to.reset_index().rename(columns={'dest': 'bts_loc_code', 'passengers': 'p_dest'})
df_pass_to = df_pass_to.set_index(['date', 'carrier', 'bts_loc_code'])

# merge these two datasets together
df_pass_both = df_pass_from.merge(df_pass_to, how='outer', left_index=True, right_index=True)

Get number of passengers either arriving at or departing from [location], on [date], via [carrier]. 

In [122]:
# replace missing values with 0
df_pass_both = df_pass_both.replace(np.nan, 0)

# calculate the 'both' column
df_pass_both['passengers'] = df_pass_both['p_origin'] + df_pass_both['p_dest']
df_pass_both = df_pass_both.reset_index()

In [123]:
df_pass_both.head()

,date,carrier,bts_loc_code,p_origin,p_dest,passengers
0,2020-01-01,04Q,30154,5.0,8.0,13.0
1,2020-01-01,04Q,30194,2.0,2.0,4.0
2,2020-01-01,04Q,30198,4.0,0.0,4.0
3,2020-01-01,04Q,30257,2.0,2.0,4.0
4,2020-01-01,04Q,30397,6.0,6.0,12.0


In [124]:
df_pass_both.shape

(109147, 6)

### 1.4 - Create national summary entries (summing all locations together)

In [125]:
# National (all locations aggregated together) summary
df_bts_us = df_bts_region[['date', 'carrier', 'passengers']].groupby(['date', 'carrier']).sum()

In [126]:
# location 0 will designate the sum of all locations
df_bts_us['bts_loc_code'] = 0

# for the national aggregate, the same number of passengers depart and arrive
# (for in-country travel, however many go up, must come down)
df_bts_us['p_origin'] = df_bts_us['passengers']
df_bts_us['p_dest'] = df_bts_us['passengers']

# match the regional df
df_bts_us = df_bts_us[['bts_loc_code', 'p_origin', 'p_dest', 'passengers']]
df_bts_us = df_bts_us.reset_index()

# combine national data and regional data
df_bts_final = pd.concat([df_bts_us, df_pass_both])

In [127]:
df_bts_final.head()

,date,carrier,bts_loc_code,p_origin,p_dest,passengers
0,2020-01-01,04Q,0,357.0,357.0,357.0
1,2020-01-01,09Q,0,33687.0,33687.0,33687.0
2,2020-01-01,1BQ,0,700.0,700.0,700.0
3,2020-01-01,1EQ,0,1157.0,1157.0,1157.0
4,2020-01-01,1QQ,0,1490.0,1490.0,1490.0


In [128]:
df_bts_final.shape

(111941, 6)

### 1.5 - Create 'all carrier' summary entries (summing all carriers together)

In [129]:
# all carriers (all carriers aggregated together) summary
df_all_carriers = \
    df_bts_final[['date', 'bts_loc_code', 'p_origin', 'p_dest', 'passengers']].groupby(['date', 'bts_loc_code']).sum()

In [130]:
# carrier 'ALL' will designate the sum of all carriers
df_all_carriers['carrier'] = 'ALL'

# match the format of our final df
df_all_carriers = df_all_carriers[['carrier', 'p_origin', 'p_dest', 'passengers']]
df_all_carriers = df_all_carriers.reset_index()

# combine all carrier data with other data
df_bts_final = pd.concat([df_all_carriers, df_bts_final])

In [131]:
df_bts_final.head()

,date,bts_loc_code,carrier,p_origin,p_dest,passengers
0,2020-01-01,0,ALL,61848685.0,61848685.0,61848685.0
1,2020-01-01,30006,ALL,4.0,4.0,8.0
2,2020-01-01,30007,ALL,687.0,655.0,1342.0
3,2020-01-01,30009,ALL,2739.0,2503.0,5242.0
4,2020-01-01,30011,ALL,813.0,823.0,1636.0


In [132]:
df_bts_final.shape

(138450, 6)

### 1.6 - Write our finalized travel data to file

In [133]:
df_bts_final.to_pickle('full_BTS_data.pkl', protocol=4)

# 2.0 - Covid data

### 2.1 - Obtaining and reading in the raw data

We obtained our Covid data from Covid Act Now (https://covidactnow.org).  The data is available to download after registering for a free API key at https://covidactnow.org/data-api.

In order to remove my personal API key from this notebook, I added code to write the result out to a pickle file, then read from that pickle file to carry on with the notebook.  However, I am leaving the original code, commented out, so someone can recreate what we've done with their own API key.

In [134]:
# URLs and API key to retrieve COVID Act Now Data
#api_key = [your API key here]
#us_data = 'https://api.covidactnow.org/v2/country/US.timeseries.csv?apiKey='
#cbsa_data = 'https://api.covidactnow.org/v2/cbsas.timeseries.csv?apiKey='

In [135]:
# Read data at the national and regional (CBSA) levels, limiting to needed columns
#df_us_covid = pd.read_csv(us_data+api_key)[['date','fips','actuals.newCases','actuals.newDeaths']]
#df_cbsa_covid = pd.read_csv(cbsa_data+api_key)[['date','fips','actuals.newCases','actuals.newDeaths']]

In [136]:
# write to pickle
#df_us_covid.to_pickle('df_us_covid_data.pkl', protocol=4)
#df_cbsa_covid.to_pickle('df_cbsa_covid_data.pkl', protocol=4)

In [137]:
# read from pickle
df_us_covid = pd.read_pickle('df_us_covid_data.pkl')
df_cbsa_covid = pd.read_pickle('df_cbsa_covid_data.pkl')

In [138]:
# There are some null and zero values in the data but these will be addressed
# when aggregated by month
df_us_covid.head()

,date,fips,actuals.newCases,actuals.newDeaths
0,2020-03-09,0,NaN,NaN
1,2020-03-10,0,NaN,NaN
2,2020-03-11,0,NaN,NaN
3,2020-03-12,0,403.0,6.0
4,2020-03-13,0,542.0,7.0


In [139]:
df_us_covid.shape

(1077, 4)

In [140]:
# Again, there are some null and zero values as particular locations may
# not have reported statistics on a given day
df_cbsa_covid.head()

,date,fips,actuals.newCases,actuals.newDeaths
0,2020-05-08,10100,NaN,NaN
1,2020-05-09,10100,NaN,NaN
2,2020-05-10,10100,NaN,NaN
3,2020-05-11,10100,NaN,NaN
4,2020-05-12,10100,NaN,NaN


In [141]:
df_cbsa_covid.shape

(1008729, 4)

### 2.2 - Organize the covid data, aggregating to months in order to match the travel data

In [142]:
# function to prep covid data
def process_covid_df(df):
    # Convert date string to datetime
    df['date'] = pd.to_datetime(df.date, format='%Y-%m-%d')
    
    # Group by month
    df = df.groupby([df.date.dt.year,df.date.dt.month,df.fips]).sum(numeric_only=True)
    
    # Reorganize index and columns
    df.index.names = ['year','month','fips']
    df.columns = ['cases','deaths']
    df.reset_index(inplace=True)
    df = df[df.year<2023]
    df['date'] = pd.to_datetime(dict(year=df.year, month=df.month, day=1))
    df.index = [df.date, df.fips]
    df.drop(['year','month','date', 'fips'], axis=1, inplace=True)
    df = df[(df.cases>0) | (df.deaths>0)]
    
    return df

In [143]:
# Prepare covid data at the monthly level
df_us_covid = process_covid_df(df_us_covid)
df_cbsa_covid = process_covid_df(df_cbsa_covid)

In [144]:
# combine national and regional covid dfs
df_covid = pd.concat([df_us_covid, df_cbsa_covid])
df_covid = df_covid.reset_index().set_index('date')

In [145]:
df_covid.head()

,fips,cases,deaths
date,,,
2020-03-01,0,184104.0,4199.0
2020-04-01,0,883830.0,55938.0
2020-05-01,0,721682.0,38464.0
2020-06-01,0,820785.0,18498.0
2020-07-01,0,1916317.0,24599.0


In [146]:
df_covid.shape

(31566, 3)

### 2.3 - Write our finalized covid data to file

In [147]:
# Write files as pickles for future use
df_covid.to_pickle('covid_data.pkl', protocol=4)

# 3.0 - Location join information

The Bureau of Transportation Statistics and the Covid Act Now Coalition use different codes for locations.  We manually created a lookup table compiling the codes of 40 locations in order to join the covid data and travel data for those locations. (We trimmed two of the locations due to missing data in COVID statistics and added a summary row to cover the national-level data.)

In [148]:
df_loc_join = pd.read_csv('code_lookup.csv')

In [149]:
df_loc_join.head()

,Market Name,Code,Market,CBSA
0,USA,USA,0,0
1,"Atlanta, GA",ATL,30397,12060
2,"Dallas/Fort Worth, TX",DFW,30194,19100
3,"New York City, NY",NYC,31703,35620
4,"Chicago, IL",CHI,30977,16980


In [150]:
df_loc_join.shape

(39, 4)

# 4.0 - Join the travel data and covid data

### 4.1 - Get the subset of travel data as specified by our location selection

In [151]:
# get unique list of travel locations
list_travel_loc = df_loc_join['Market'].unique()

# get subset of travel data that we're looking for
df_selected_travel = df_bts_final.loc[df_bts_final['bts_loc_code'].isin(list_travel_loc)].copy()

In [152]:
df_selected_travel.head()

,date,bts_loc_code,carrier,p_origin,p_dest,passengers
0,2020-01-01,0,ALL,61848685.0,61848685.0,61848685.0
26,2020-01-01,30194,ALL,3011461.0,3021309.0,6032770.0
27,2020-01-01,30198,ALL,318960.0,318072.0,637032.0
50,2020-01-01,30325,ALL,2395554.0,2383739.0,4779293.0
57,2020-01-01,30397,ALL,3532810.0,3550718.0,7083528.0


In [153]:
df_selected_travel.shape

(31481, 6)

### 4.2 - Add join information (location name and 3-char code) for each BTS location code

In [154]:
# for each unique bts location code
for bts_loc in list_travel_loc:
    # get the location name
    loc_name = df_loc_join.loc[df_loc_join['Market']==bts_loc]['Market Name'].iloc[0]
    df_selected_travel.loc[df_selected_travel['bts_loc_code'] == bts_loc, 'location'] = loc_name
    
    # get the 3-char location code
    loc_code = df_loc_join.loc[df_loc_join['Market']==bts_loc]['Code'].iloc[0]
    df_selected_travel.loc[df_selected_travel['location'] == loc_name, 'loc_code'] = loc_code
    
# reorder columns, drop bts_loc_code    
df_selected_travel = df_selected_travel[['date', 'location', 'loc_code', 'carrier', 'p_origin', 'p_dest', 'passengers']]

In [155]:
df_selected_travel.head()

,date,location,loc_code,carrier,p_origin,p_dest,passengers
0,2020-01-01,USA,USA,ALL,61848685.0,61848685.0,61848685.0
26,2020-01-01,"Dallas/Fort Worth, TX",DFW,ALL,3011461.0,3021309.0,6032770.0
27,2020-01-01,"Pittsburgh, PA",PIT,ALL,318960.0,318072.0,637032.0
50,2020-01-01,"Denver, CO",DEN,ALL,2395554.0,2383739.0,4779293.0
57,2020-01-01,"Atlanta, GA",ATL,ALL,3532810.0,3550718.0,7083528.0


In [156]:
df_selected_travel.shape

(31481, 7)

### 4.3 - Get the subset of covid data as specified by our location selection

In [157]:
# get unique list of covid locations
covid_loc_list = df_loc_join['CBSA'].unique()

# get subset of covid data that we're looking for
df_final_covid = df_covid.loc[df_covid['fips'].isin(covid_loc_list)]

# rename fips column
df_final_covid = df_final_covid.rename(columns={'fips': 'covid_loc_code'})

In [158]:
df_final_covid.head()

,covid_loc_code,cases,deaths
date,,,
2020-03-01,0,184104.0,4199.0
2020-04-01,0,883830.0,55938.0
2020-05-01,0,721682.0,38464.0
2020-06-01,0,820785.0,18498.0
2020-07-01,0,1916317.0,24599.0


In [159]:
df_final_covid.shape

(1322, 3)

### 4.4 - Add join information (location name and 3-char code) for each covid location code

In [160]:
# replace location, add loc
for cov_loc in covid_loc_list:
    # get the location name
    loc_name = df_loc_join.loc[df_loc_join['CBSA']==cov_loc]['Market Name'].iloc[0]
    df_final_covid.loc[df_final_covid['covid_loc_code'] == cov_loc, 'location'] = loc_name
    
    # get the 3-char location code
    loc_code = df_loc_join.loc[df_loc_join['CBSA']==cov_loc]['Code'].iloc[0]
    df_final_covid.loc[df_final_covid['covid_loc_code'] == cov_loc, 'loc_code'] = loc_code
    
# reorder columns, drop covid_loc_code      
df_final_covid = df_final_covid[['location', 'loc_code', 'cases', 'deaths']]

In [161]:
df_final_covid.head()

,location,loc_code,cases,deaths
date,,,,
2020-03-01,USA,USA,184104.0,4199.0
2020-04-01,USA,USA,883830.0,55938.0
2020-05-01,USA,USA,721682.0,38464.0
2020-06-01,USA,USA,820785.0,18498.0
2020-07-01,USA,USA,1916317.0,24599.0


In [162]:
df_final_covid.shape

(1322, 4)

### 4.5 - Join covid data with travel data on location join data

In [163]:
# final merge
df_final_merged = pd.merge(df_selected_travel.reset_index(), df_final_covid.reset_index(), 
                           left_on=['date', 'location', 'loc_code'], right_on=['date', 'location', 'loc_code'])

# set columns
df_final_merged = df_final_merged[['date', 'location', 'loc_code', 'carrier', 
                                   'p_origin', 'p_dest', 'passengers', 
                                   'cases', 'deaths']]

In [164]:
df_final_merged.head()

,date,location,loc_code,carrier,p_origin,p_dest,passengers,cases,deaths
0,2020-03-01,USA,USA,ALL,34527687.0,34527687.0,34527687.0,184104.0,4199.0
1,2020-03-01,USA,USA,04Q,274.0,274.0,274.0,184104.0,4199.0
2,2020-03-01,USA,USA,09Q,26506.0,26506.0,26506.0,184104.0,4199.0
3,2020-03-01,USA,USA,1EQ,637.0,637.0,637.0,184104.0,4199.0
4,2020-03-01,USA,USA,1QQ,417.0,417.0,417.0,184104.0,4199.0


In [165]:
df_final_merged.shape

(29479, 9)

### 4.6 - Write our finalized dataframe to file

In [166]:
df_final_merged.to_pickle('Compiled_data.pkl', protocol=4)

# 5.0 - Profile data, all locations aggregated, all airlines aggregated

In [167]:
usa_df = df_final_merged.loc[(df_final_merged['loc_code']=='USA') & (df_final_merged['carrier']=='ALL')]

In [168]:
profile_report = ProfileReport(usa_df, title='Covid/Travel Data Profile Report')

In [169]:
profile_report.to_file('COVID_Travel_US_Profile.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]